In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from sympy import false
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
model = nn.Sequential(

    # Block 1
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 32×32 → 16×16

    # Block 2
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 16×16 → 8×8

    # Block 3
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 8×8 → 4×4

    # Block 4
    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.Flatten(),

    nn.Linear(256 * 4 * 4, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 100)   # CIFAR-100
)


In [ ]:
def make_model():
    return nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),      # 32×32 → 16×16

        # Block 2
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),      # 16×16 → 8×8

        # Block 3
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),      # 8×8 → 4×4

        # Block 4
        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),

        nn.Flatten(),

        nn.Linear(256 * 4 * 4, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 100)   # CIFAR-100
    )


In [ ]:
def set_seed(seed):
    import random, numpy as np, torch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
def plot_training_curves(results_dict):
    plt.figure(figsize=(12,5))

    # ---- Training Loss ----
    plt.subplot(2,2,1)
    for name, history in results_dict.items():
        plt.plot(history["train_loss"], label=f"{name} Train Loss")
    plt.title("Training Loss Comparison")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(2,2,2)
    for name, history in results_dict.items():
        plt.plot(history["train_acc"], label=f"{name} Train Accuracy")
    plt.title("Training Accuracy Comparison")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    # ---- Validation Accuracy ----
    plt.subplot(2,2,3)
    for name, history in results_dict.items():
        plt.plot(history["val_loss"], label=f"{name} Val Loss")
    plt.title("Validation Loss Comparison")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(2,2,4)
    for name, history in results_dict.items():
        plt.plot(history["val_acc"], label=f"{name} Val Acc")
    plt.title("Validation Accuracy Comparison")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# 3. CIFAR-100 DATASET

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])


train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# 4. Ορισμός συναρτήσεων κόστους και βελτιστοποίησης
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# --------------------------------------------------------
# 5. Training
# --------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = make_model().to(device)

def train_one(model, train_loader, test_loader, criterion, optimizer, device, epochs):
    model.to(device)

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
    }

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct, total = 0, 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} [Train]",leave=False)

        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs,1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            train_bar.set_postfix(
                loss=loss.item(),
                accuracy=100 * correct / total
            )

        train_loss = running_loss / len(train_loader)
        train_acc = 100*correct/total
        val_loss, val_acc = evaluate(model, test_loader, criterion, device, epoch, epochs)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        print(f"Epoch {epoch+1}/{epochs} "
          f"| TrainLoss: {train_loss:.4f} "
          f"| ValLoss: {val_loss:.4f} "
          f"| TrainAcc: {train_acc:.2f}% "
          f"| ValAcc: {val_acc:.2f}%")

    return history



In [ ]:
def evaluate(model, loader, criterion, device, epoch=None, epochs=None):
    model.eval()
    correct, total = 0,0
    running_loss = 0

    desc = "Validation" if epoch is None else f"Epoch {epoch+1}/{epochs} [Val]"
    val_bar = tqdm(loader, desc=desc, leave=False)

    with torch.no_grad():
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _, preds = torch.max(outputs,1)
            correct += (preds==labels).sum().item()
            total += labels.size(0)

            val_bar.set_postfix(
                loss=loss.item(),
                accuracy=100 * correct / total
            )

    val_loss = running_loss / len(loader)
    val_acc = 100*correct/total
    return val_loss, val_acc

In [ ]:
def show_predictions(model, loader, device, class_names, n=8):
    model.eval()

    images, labels = next(iter(loader))
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

    images = images.cpu()
    labels = labels.cpu()
    preds = preds.cpu()

    plt.figure(figsize=(16, 6))

    for i in range(n):
        plt.subplot(2, n//2, i+1)

        # unnormalize
        img = images[i].permute(1,2,0)
        img = img * torch.tensor([0.2675, 0.2565, 0.2761]) + torch.tensor([0.5071, 0.4867, 0.4408])
        img = torch.clamp(img, 0, 1)

        plt.imshow(img)
        true_label = class_names[labels[i]]
        pred_label = class_names[preds[i]]

        color = "green" if labels[i] == preds[i] else "red"

        plt.title(f"T: {true_label}\nP: {pred_label}", color=color, fontsize=10)
        plt.axis("off")

    plt.suptitle("Classification results (Green=Correct, Red=Wrong)", fontsize=14)
    plt.show()


In [ ]:
lrs = [1e-2, 1e-3, 1e-4, 1e-5]
results = {}
for lr in lrs:
    print(f"Training with LR={lr}")
    set_seed(0)
    model = make_model()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    history = train_one(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)

    results[lr] = history

plot_training_curves(results)

In [ ]:
from torch.optim import  Adam, SGD, RMSprop

results = {}
optimizers = [Adam, SGD, RMSprop]
for opt_cls in optimizers:
    name = opt_cls.__name__
    print(f"Training with {name}")
    set_seed(0)
    model = make_model().to(device)
    optimizer = opt_cls(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    history = train_one(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)

    results[name] = history

plot_training_curves(results)

In [ ]:
losses = {
    "CrossEntropy": nn.CrossEntropyLoss(),
    "LabelSmoothing0.1":nn .CrossEntropyLoss(label_smoothing=0.1)
}
results = {}
for name, loss_fn in losses.items():
    print(f"Training with {name}")
    set_seed(0)
    model = make_model().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    history = train_one(model, train_loader, test_loader, loss_fn, optimizer, device, epochs=10)
    results[name] = history


plot_training_curves(results)


In [ ]:
# Ενδεικτικά Αποτελέσματα Ταξινόμησης
class_names = test_dataset.classes
show_predictions(model, test_loader, device, class_names, n=8)
